In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from helpers.utils import creat_datasets, creating_val_data, reset_graph, grid_serach, read_pickle_file, write_pickle_file
from helpers.utils import creating_train_val_test_datasets, tensorboard_callback,  save_best_model_callback
from helpers.bln_layer import  bln_layer
from helpers.dense_layer import  dense_layer
from helpers.bln_callback import bln_callback 




from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

In [2]:
tf.config.experimental_run_functions_eagerly(True)

In [3]:
tf.__version__

'2.1.0'

### Loading data

In [4]:
random_seed = 100
minibatch = 25
buffersize = 60000
number_valid_sampels = 5000 # number of validation data
epochs = 5
learning_rate = 0.005

#((25000  - 5000 )/ 25 ) * .2
number_batches_train = 160 # number of batches to train, each batch of size minibatch parameter
number_batches_valid = 40 # number of batches to validate, each batch of size minibatch parameter (( 5000 )/ 25 ) * .2
num_classes = 1

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)


In [5]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...


C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Rima\.conda\envs\bln\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [6]:
x_train, y_train, x_valid, y_valid = creating_val_data(x_train, y_train,
                                                       number_valid_sampels = number_valid_sampels,
                                                       random_seed=random_seed)

train_dataset, valid_dataset, test_dataset = creating_train_val_test_datasets(x_train, y_train,
                                                                              x_test, y_test,
                                                                              x_valid, y_valid, 
                                                                              minibatch = minibatch,
                                                                              buffersize= buffersize,
                                                                              random_seed=random_seed)

In [7]:
train_dataset

<PrefetchDataset shapes: ((25, 80), (25,)), types: (tf.int32, tf.int64)>

# 1. Using Batch Layer Normalization Layer

In [8]:
def BLNLayer_model(inputshape= (80), max_features = 20000, embed_size=256, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1,
                          batch_size = 10,
                          b_mm = True, b_mv = True,
                          f_mm = False, f_mv = False):
    
   
    # building the model
  
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn1', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2,activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    
    
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn2', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x = bln_layer(stateful = True, batchsize= batch_size, name = 'bn3', 
                  batch_moving_mean = b_mm, batch_moving_var = b_mv,
                  feature_moving_mean = f_mm, feature_moving_var = f_mv)(x)
           
    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])

In [9]:
model_bln_layer = BLNLayer_model(inputshape= (80), max_features = max_features, 
                                        embed_size= 256, random_seed = random_seed,
                                        lstm_unit = 128 , dense_units= num_classes,
                                        batch_size = minibatch,
                                        b_mm = False, b_mv = False,
                                        f_mm = False, f_mv = False
                                        )



In [10]:
model_bln_layer.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 256)             5120000   
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             197120    
_________________________________________________________________
bn1 (bln_layer)              (25, 80, 128)             24738     
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
bn2 (bln_layer)              (25, 64)                  308       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

# callbacks

In [11]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bln_layer_imdb_batchSize_' + str(minibatch)
 
save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

# Callback for resetting moving mean and variances at the end of each epoch
bln_layer_cb = bln_callback()

bln_layer_cb_list = [save_bm_cb, tb_cb, bln_layer_cb]

In [12]:
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)

model_bln_layer.compile(optimizer = adam,
                               loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                               metrics = [tf.keras.metrics.BinaryAccuracy()])

In [13]:
model_bln_layer_history =  model_bln_layer.fit(train_dataset.take(number_batches_train), epochs=epochs,
                                                verbose=1, callbacks=bln_layer_cb_list,
                                                validation_data=valid_dataset.take(number_batches_valid),
                                                shuffle=True)

Train for 160 steps, validate for 40 steps
Epoch 1/5
160/160 [==============================] - 129s 808ms/step - loss: 0.6438 - binary_accuracy: 0.5725 - val_loss: 0.5718 - val_binary_accuracy: 0.6870
Epoch 2/5
160/160 [==============================] - 135s 842ms/step - loss: 0.3874 - binary_accuracy: 0.8215 - val_loss: 0.6508 - val_binary_accuracy: 0.7280
Epoch 3/5
160/160 [==============================] - 133s 832ms/step - loss: 0.1897 - binary_accuracy: 0.9230 - val_loss: 0.8240 - val_binary_accuracy: 0.7120
Epoch 4/5
160/160 [==============================] - 134s 838ms/step - loss: 0.0937 - binary_accuracy: 0.9657 - val_loss: 0.9585 - val_binary_accuracy: 0.7240
Epoch 5/5
160/160 [==============================] - 138s 861ms/step - loss: 0.0648 - binary_accuracy: 0.9755 - val_loss: 1.1418 - val_binary_accuracy: 0.7260


In [14]:
del model_bln_layer, save_bm_cb, tb_cb, bln_layer_cb, bln_layer_cb_list
reset_graph()

session is clear


In [15]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bln_layer_imdb_batchSize_25/best_model.h5'

In [16]:
save_eval_path = "./logs/" + folder_name + '/'+ str(number_batches_train) +"_sorted_evaluation.pkl"
evaluation = grid_serach(BLNLayer_model, test_dataset,
                         batch_size = minibatch, sort=True,
                         save_eval_path = save_eval_path,
                         weights_path = weights_path,
                         loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                         metrics = tf.keras.metrics.BinaryAccuracy())



1000/1000 [==============================] - 298s 298ms/step - loss: 0.5972 - binary_accuracy: 0.7449
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.5971591956093907, 0.74492]}
session is clear
1000/1000 [==============================] - 300s 300ms/step - loss: 0.5926 - binary_accuracy: 0.7450
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.5971591956093907, 0.74492], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.5925961168557405, 0.74496]}
session is clear
1000/1000 [==============================] - 298s 298ms/step - loss: 0.5969 - binary_accuracy: 0.7450
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.5971591956093907, 0.74492], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.5925961168557405, 0.74496], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.596867555141449, 0.74496]}
session is clear
1000/1000 [==============================] - 298s 298ms/step - loss: 0.5924 - binary_accuracy: 0.7449
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.5971591956093907, 0.74492], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.59

1000/1000 [==============================] - 300s 300ms/step - loss: 0.5709 - binary_accuracy: 0.6901
{'Bmm_True Bmv_True Fmm_True Fmv_True': [0.5971591956093907, 0.74492], 'Bmm_True Bmv_True Fmm_True Fmv_False': [0.5925961168557405, 0.74496], 'Bmm_True Bmv_True Fmm_False Fmv_True': [0.596867555141449, 0.74496], 'Bmm_True Bmv_True Fmm_False Fmv_False': [0.5923926493823528, 0.74492], 'Bmm_True Bmv_False Fmm_True Fmv_True': [0.5305076062381268, 0.7416], 'Bmm_True Bmv_False Fmm_True Fmv_False': [0.5308341007232666, 0.74152], 'Bmm_True Bmv_False Fmm_False Fmv_True': [0.530445402622223, 0.74172], 'Bmm_True Bmv_False Fmm_False Fmv_False': [0.5307774426937103, 0.7416], 'Bmm_False Bmv_True Fmm_True Fmv_True': [0.5894799150526524, 0.71368], 'Bmm_False Bmv_True Fmm_True Fmv_False': [0.5874100328683853, 0.71416], 'Bmm_False Bmv_True Fmm_False Fmv_True': [0.5892977966964245, 0.7136], 'Bmm_False Bmv_True Fmm_False Fmv_False': [0.5872673473656177, 0.71408], 'Bmm_False Bmv_False Fmm_True Fmv_True': [

In [17]:
evaluation

[('Bmm_True Bmv_False Fmm_False Fmv_True', [0.530445402622223, 0.74172]),
 ('Bmm_True Bmv_False Fmm_True Fmv_True', [0.5305076062381268, 0.7416]),
 ('Bmm_True Bmv_False Fmm_False Fmv_False', [0.5307774426937103, 0.7416]),
 ('Bmm_True Bmv_False Fmm_True Fmv_False', [0.5308341007232666, 0.74152]),
 ('Bmm_False Bmv_False Fmm_False Fmv_True', [0.5708688415884972, 0.69012]),
 ('Bmm_False Bmv_False Fmm_True Fmv_True', [0.5708734404444694, 0.68996]),
 ('Bmm_False Bmv_False Fmm_False Fmv_False', [0.5713950329124927, 0.68996]),
 ('Bmm_False Bmv_False Fmm_True Fmv_False', [0.571403653472662, 0.69008]),
 ('Bmm_False Bmv_True Fmm_False Fmv_False', [0.5872673473656177, 0.71408]),
 ('Bmm_False Bmv_True Fmm_True Fmv_False', [0.5874100328683853, 0.71416]),
 ('Bmm_False Bmv_True Fmm_False Fmv_True', [0.5892977966964245, 0.7136]),
 ('Bmm_False Bmv_True Fmm_True Fmv_True', [0.5894799150526524, 0.71368]),
 ('Bmm_True Bmv_True Fmm_False Fmv_False', [0.5923926493823528, 0.74492]),
 ('Bmm_True Bmv_True Fmm_T

# 2.Using  Batch Normalization implemented in Keras

In [18]:
def bn_keras_model(inputshape= (80), max_features = 20000, embed_size=256, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1,
                          batch_size = 60):
    

    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn1') (x) 
    
    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn2') (x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x =  tf.keras.layers.BatchNormalization(momentum = 0.99,  name = 'bn3') (x) 

    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])


In [19]:
model_bn_keras = bn_keras_model(inputshape= (80), max_features = max_features, 
                                embed_size= 256, random_seed = random_seed,
                                lstm_unit = 128 , dense_units= num_classes,
                                batch_size = minibatch)
model_bn_keras.summary()

#### Compiling 
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)
model_bn_keras.compile(optimizer = adam,
                       loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics = [tf.keras.metrics.BinaryAccuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 256)             5120000   
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             197120    
_________________________________________________________________
bn1 (BatchNormalization)     (25, 80, 128)             512       
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
bn2 (BatchNormalization)     (25, 64)                  256       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

In [20]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/bn_Keras_imdb_batchSize_' + str(minibatch)

save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)


bn_keras_cb_list = [save_bm_cb, tb_cb]

In [21]:
model_bn_keras_history =  model_bn_keras.fit(train_dataset.take(number_batches_train),
                                             epochs = epochs, verbose=1, 
                                             callbacks = bn_keras_cb_list,
                                             validation_data = valid_dataset.take(number_batches_valid),
                                             shuffle = True)

Train for 160 steps, validate for 40 steps
Epoch 1/5
160/160 [==============================] - 133s 829ms/step - loss: 0.7066 - binary_accuracy: 0.5238 - val_loss: 0.6666 - val_binary_accuracy: 0.4880
Epoch 2/5
160/160 [==============================] - 132s 824ms/step - loss: 0.5074 - binary_accuracy: 0.7287 - val_loss: 0.6424 - val_binary_accuracy: 0.6360
Epoch 3/5
160/160 [==============================] - 130s 815ms/step - loss: 0.3155 - binary_accuracy: 0.8543 - val_loss: 0.8765 - val_binary_accuracy: 0.6090
Epoch 4/5
160/160 [==============================] - 132s 825ms/step - loss: 0.2473 - binary_accuracy: 0.8985 - val_loss: 1.0465 - val_binary_accuracy: 0.6300
Epoch 5/5
160/160 [==============================] - 131s 821ms/step - loss: 0.1971 - binary_accuracy: 0.9260 - val_loss: 0.9727 - val_binary_accuracy: 0.6810


In [22]:
# Select the model with the lowest validation loss
weights_path = './models/rnn/bn_Keras_imdb_batchSize_25/best_model.h5'
model_bn_keras.load_weights(weights_path)

In [23]:
model_bn_keras_test_history = model_bn_keras.evaluate(test_dataset)

1000/1000 [==============================] - 296s 296ms/step - loss: 0.6393 - binary_accuracy: 0.6297


In [24]:
del model_bn_keras, save_bm_cb, tb_cb, bn_keras_cb_list 
reset_graph()

session is clear


# 3.Using  Layer normalization  implemented in Keras


In [25]:
def ln_keras_model(inputshape= (80), max_features = 20000, embed_size=256, random_seed = 100,
                          lstm_unit = 128 , dense_units= 1,
                          batch_size = 60):
    

    
    input_lyr = tf.keras.Input(shape = inputshape, batch_size=batch_size, name = 'input')
    
    x = Embedding(max_features, embed_size,
                  embeddings_initializer=tf.keras.initializers.GlorotUniform(seed=random_seed))(input_lyr)
    
    x = LSTM(lstm_unit, dropout=0.2, recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=True)(x)
    x =  tf.keras.layers.LayerNormalization()(x) 

    x = LSTM(lstm_unit//2, dropout=0.2,  recurrent_dropout=0.2, activation='tanh',
            recurrent_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed),
            kernel_initializer = tf.keras.initializers.GlorotUniform(seed=random_seed) ,
            return_sequences=False)(x)
    x =  tf.keras.layers.LayerNormalization()(x) 
    
    x = dense_layer(units = 32, name = 'dense1', random_seed=random_seed)(x)
    x = tf.keras.layers.Activation('tanh')(x)
    x =  tf.keras.layers.LayerNormalization()(x) 

    
    output_lyr = dense_layer(units = dense_units, name = 'dense2', random_seed=random_seed)(x)    
    return tf.keras.Model(inputs = [input_lyr], outputs = [output_lyr])



In [26]:
model_ln_keras = ln_keras_model(inputshape= (80), max_features = max_features, 
                                embed_size= 256, random_seed = random_seed,
                                lstm_unit = 128 , dense_units= num_classes,
                                batch_size = minibatch)
model_ln_keras.summary()

#### Compiling 
adam = tf.keras.optimizers.Adam(learning_rate= learning_rate, beta_1=.9, beta_2=.999, epsilon=1e-08)

model_ln_keras.compile(optimizer = adam,
                       loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metrics = [tf.keras.metrics.BinaryAccuracy()])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(25, 80)]                0         
_________________________________________________________________
embedding (Embedding)        (25, 80, 256)             5120000   
_________________________________________________________________
lstm (LSTM)                  (25, 80, 128)             197120    
_________________________________________________________________
layer_normalization (LayerNo (25, 80, 128)             256       
_________________________________________________________________
lstm_1 (LSTM)                (25, 64)                  49408     
_________________________________________________________________
layer_normalization_1 (Layer (25, 64)                  128       
_________________________________________________________________
dense1 (dense_layer)         (25, 32)                  2080  

In [27]:
# Callbacks for saving best model and tensorboard
folder_name = '/rnn/ln_Keras_imdb_batchSize_' + str(minibatch) 

save_bm_cb = save_best_model_callback(folder_name)
tb_cb = tensorboard_callback(folder_name)

ln_keras_cb_list = [save_bm_cb, tb_cb]

In [28]:
model_ln_keras_history =  model_ln_keras.fit(train_dataset.take(number_batches_train),
                                             epochs=epochs, verbose=1, 
                                             callbacks=ln_keras_cb_list,
                                             validation_data=valid_dataset.take(number_batches_valid),
                                             shuffle=True)

Train for 160 steps, validate for 40 steps
Epoch 1/5
160/160 [==============================] - 131s 820ms/step - loss: 0.7054 - binary_accuracy: 0.4970 - val_loss: 0.7082 - val_binary_accuracy: 0.4880
Epoch 2/5
160/160 [==============================] - 132s 826ms/step - loss: 0.6956 - binary_accuracy: 0.4985 - val_loss: 0.7078 - val_binary_accuracy: 0.4880
Epoch 3/5
160/160 [==============================] - 132s 825ms/step - loss: 0.6740 - binary_accuracy: 0.5332 - val_loss: 0.6716 - val_binary_accuracy: 0.6330
Epoch 4/5
160/160 [==============================] - 129s 805ms/step - loss: 0.4677 - binary_accuracy: 0.7632 - val_loss: 0.7473 - val_binary_accuracy: 0.6920
Epoch 5/5
160/160 [==============================] - 130s 810ms/step - loss: 0.2513 - binary_accuracy: 0.9000 - val_loss: 0.7424 - val_binary_accuracy: 0.7170


In [29]:
# Select the model with the lowest validation loss

weights_path = './models/rnn/ln_Keras_imdb_batchSize_25/best_model.h5'

model_ln_keras.load_weights(weights_path)

In [30]:
model_ln_keras_test_history = model_ln_keras.evaluate(test_dataset)

1000/1000 [==============================] - 300s 300ms/step - loss: 0.6884 - binary_accuracy: 0.6393


In [31]:
del model_ln_keras,save_bm_cb, ln_keras_cb_list
reset_graph()

session is clear


# 4.writing pickle files

In [32]:
write_pickle_file('./logs/rnn/ln_Keras_imdb_batchSize_25/model_ln_keras_history_25.pickle', model_ln_keras_history.history)
write_pickle_file('./logs/rnn/ln_Keras_imdb_batchSize_25/model_ln_keras_test_history_25.pickle', model_ln_keras_test_history)

write_pickle_file('./logs/rnn/bn_Keras_imdb_batchSize_25/model_bn_keras_history_25.pickle', model_bn_keras_history.history)
write_pickle_file('./logs/rnn/bn_Keras_imdb_batchSize_25/model_bn_keras_test_history_25.pickle', model_bn_keras_test_history)

write_pickle_file('./logs/rnn/bln_layer_imdb_batchSize_25/model_bln_keras_history_25.pickle', model_bln_layer_history.history)
write_pickle_file('./logs/rnn/bln_layer_imdb_batchSize_25/model_bln_keras_test_history_25.pickle', evaluation)